<a href="https://colab.research.google.com/github/JSJeong-me/Generate_AI_for_Image/blob/main/99-StableDiffusionWebUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###이 Google Colab은 필요에 따라 업데이트 됩니다. 최신 버전은 [이곳](https://colab.research.google.com/github/DigiClau/stablediffusion_webui/blob/main/StableDiffusionWebUI_digiclau.ipynb)에 있습니다.

###  [Stable Diffusion WebUI Forge Colab도 있습니다.](https://colab.research.google.com/github/DigiClau/stablediffusion_webui/blob/main/StableDiffusionWebUI_digiclau_Forge.ipynb)
###  [ComfyUI Colab도 있습니다.](https://colab.research.google.com/github/DigiClau/stablediffusion_webui/blob/main/ComfyUI_digiclau.ipynb)


## Instruction(필수 사항)
In the next cell...
1. 사용할 모델들을 선택합니다.
2. 외쪽에 있는 Play버튼을 눌러서 시작합니다.
3. 모든 설치가 끝날때까지 기다립니다. 시간이 걸립니다.
4. 설치가 끝난후, public링크를 누르면, 새로운 웹브라우저 창에서 Stable Diffusion 이 시작됩니다.
5. 더 자세한 내용은, 위에 있는 사용 설명 영상을 보시기 바랍니다.

## ngrok (선택 사항)
AUTOMATIC1111은 Gradio 라이브러리를 사용해서, Web UI를 제공합니다. 하지만, Google Colab에서 사용시, 접속이 끊기거나, 반응하지 않는 경우가 종종(혹은 자주) 발생합니다.\
[ngrok](https://ngrok.com/)을 사용하면 그러한 문제를 방지할 수 있습니다.  Google Colab에서 Stable Diffusion을 사용하시다가, WebUI 사용에 문제가 발생한다면 [ngrok](https://ngrok.com/)을 사용하시기 바랍니다.\
ngrok 사용방법:
  1. ngrok 웹사이트로 갑니다. https://ngrok.com/
  2. ngrok 계좌를 만듭니다. 무료입니다.
  3. ngrok 웹사이트에서 Authtoken을 만듭니다.
  4. Authtoken을 복사해서, 아래에 있는 ngrok란에 붙여줍니다.

## ### 최근 업데이트 정보
- 2024년 3월 10일:
 - AUTOMATIC1111 1.8.0 버전
 - 오래된 모델들 삭제
- 2024년 1월:
 - Replacer (Segment Anything 포함) 확장 프로그램 추가
 - AnimateDiff 새로운 모델들 추가
 - AUTOMATIC1111 1.7.0 버전
 - Inpaint Anything 확장 프로그램 추가
- 2023년:
 - Reactor 추가
 - ControlNet SDXL모델들 추가
 - SadTalker 확장 프로그램 추가  
 - LoopbackWave 스크립트 추가
 - DWPose 선행처리기 추가
 - Dynamic Thresholding (CFG Scale Fix) 확장 프로그램 추가
 - AnimateDiff 확장 프로그램 추가
 - 커스텀모델 사용 옵션 (원하는 Checkpoint모델 추가 사용 가능)
 - SDXL 1.0 base/refiner 모델 추가
- 2023년 7월 25일: AUTOMATIC1111 1.4.1 버전으로 시작


In [ ]:
import os, subprocess, time, glob
#@title <font size="6" color="black">Stable Diffusion 시작하는 섹션</font>
output_path = 'digiclau_lab'

#@markdown ## 🔩🔩🔩🔩🔩🔩🔩🔩🔩🔩🔩🔩🔩🔩🔩🔩🔩🔩🔩🔩🔩🔩🔩🔩🔩
#@markdown ## (선택사항) ngrok Authtoken을 이곳에 넣으면, ngrok GUI가 사용됩니다..
#@markdown ### 이곳을 공백으로 남겨두면, Gradio GUI가 기본으로 사용됩니다.
ngrok ='' #@param {type: "string"}

#@markdown ## 📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍
#@markdown ## AUTOMATIC1111 Stable Diffusion 버전을 선택합니다.
#@markdown ### 1.8.0 버전을 사용하면, 몇몇 확장프로그램들은 아직 업데이트가 되지 않아서 실행이 안될 수 있습니다.
#@markdown #### SD A11이 업데이트 되면, 확장프로그램들도 거기에 맞게 업데이트가 필요할 수 있습니다.
#@markdown ### (현재, DeForum, SadTalker, InfiniteZoom 등이 1.8.0과 호환 문제가 있다고 알려져 있습니다.)
version = '1.7.0' #@param ['1.8.0', '1.7.0']

#@markdown ## 🏀🏀🏀🏀🏀🏀🏀🏀🏀🏀🏀🏀🏀🏀🏀🏀🏀🏀🏀🏀🏀🏀🏀🏀🏀
#@markdown ## 구글 드라이브에 연결할지를 결정합니다.
#@markdown ### 'Yes' 를 선택하면, 구글 드라이브에 'digiclau_lab'이라는 폴더가 생기고, 그 안에 세팅 파일과, 생성된 이미지들이 저장됩니다.
#@markdown ### 'No' 를 선택하면, 구글 드라이브는 필요없지만, 세팅설정과 생성된 이미지들은 저장되지 않습니다.
use_gdrive = 'yes' #@param ['yes', 'no']
class GDriveSaveMode:
    ON = 1
    OFF = 0
if use_gdrive == 'yes':
  is_drive_used = True
elif use_gdrive == 'no':
  is_drive_used = False
else:
  raise ValueError("use_gdrive value invalid.")

#@markdown ## 🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷
#@markdown ## 아래의 Checkpoint 모델중에 사용할 모델을 선택하세요.
#@markdown ### 추가 모델은 본인의 Google Drive에 저장해서 사용할 수 있습니다.
#@markdown ### (사용할 모델이 없다면, SD 1.5 모델이 기본으로 다운로드됩니다.)
RevAnimated = False #@param{type: "boolean"}
DreamShaper_8 = False #@param{type: "boolean"}
DreamShaper_8_INPAINTING = False #@param{type: "boolean"}
PicX_real = False #@param{type: "boolean"}
CyberRealistic_V41 = True #@param{type: "boolean"}
Realistic_Vision_V60 = False #@param{type: "boolean"}
#@markdown #### SDXL 모델: SDXL 처음 사용자를 위한 [영상 보기](https://youtu.be/whn10u3kCjQ)
#@markdown ##### SDXL 1.0 모델을 사용할때는 "Change runtime type" 메뉴에서 에서 High-RAM을 선택하세요.
SDXL_10_Base_AND_Refiner = False #@param{type: "boolean"}
Juggernaut_XL_v9_RunDiffusionPhoto_v2 = False #@param{type: "boolean"}
Juggernaut_RunDiffusionPhoto2_Lightning_4Steps = False #@param{type: "boolean"}
#@markdown ##### 추가적인 다른 Checkpoint모델도 사용할 수 있습니다.
#@markdown ##### Google Drive에 파일을 저장해서 사용하는 방법을 추천합니다.
#@markdown ##### 모델 링크 주소를 아래에 넣어주세요. 여러개 일때는 콤마(,)로 구분해주세요.
#@markdown ##### (주의: Hugging Face 링크만 가능합니다.)
Model_URL = '' #@param {type: "string"}

#@markdown ## 🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷🍷
#@markdown ## 아래의 LoRA 모델중에 사용할 모델을 선택하세요.
#@markdown ### 추가 모델은 본인의 Google Drive에 저장해서 사용할 수 있습니다.
DetailTweakerLoRA = False #@param{type: "boolean"}
KoreanDollLikeness_V20 = False #@param{type: "boolean"}
#@markdown #### LCM_LoRA 설명 [영상 보기](https://youtu.be/q6srgQanVXI)
LCM_LoRA_SD15 = False #@param{type: "boolean"}
#@markdown ##### 추가적인 다른 LoRA모델도 사용할 수 있습니다.
#@markdown ##### Google Drive에 파일을 저장해서 사용하는 방법을 추천합니다.
#@markdown ##### 모델 링크 주소를 아래에 넣어주세요. 여러개 일때는 콤마(,)로 구분해주세요.
#@markdown ##### (주의: Hugging Face 링크만 가능합니다.)
LoRA_Model_URL = '' #@param {type: "string"}

#@markdown ## 🍟🍟🍟🍟🍟🍟🍟🍟🍟🍟🍟🍟🍟🍟🍟🍟🍟🍟🍟🍟🍟🍟🍟🍟🍟
#@markdown ## 아래의 확장 프로그램(Extensions)중에, 사용할 프로그램을 선택하세요.
#@markdown ### 확장 프로그램을 Google Drive에 저장하는 옵션은 제공하지 않습니다. 이 Colab 노트북을 시작할때 원하는
#@markdown ### 이 Colab Notebook을 시작할때 원하는 확장 프로그램을 선택해야 합니다.
#@markdown ### (사용할 내용만 선택하세요. 선택이 많을수록 시작하는데 시간이 더 걸립니다.)
#@markdown ### (예, ControlNet은 큰 파일들을 많이 사용합니다.)
#@markdown #### ControlNet 확장 프로그램에는 1.5모델들과 SDXL모델들이 전부 들어있습니다.
#@markdown #### ControlNet 1.5 설명 [영상 보기1](https://youtu.be/ahyGnQxP09E) [영상 보기2](https://youtu.be/XuRQJt0cowM) [영상 보기3](https://youtu.be/glNw-ANcDCI)
#@markdown #### ControlNet SDXL 설명 [영상 보기](https://youtu.be/7xuAdYFwAWA)
ControlNet = False #@param{type: "boolean"}
#@markdown #### Deforum은 ControlNet을 사용합니다.  ControNet을 함께 선택해 주세요.
Deforum = False #@param{type: "boolean"}
#@markdown #### SadTalker 설명 [영상 보기](https://youtu.be/TTr1gVhaD8k) (이 영상이 만들어진후 SadTalker가 업데이트 되어서 UI가 조금 바뀌었습니다.)
SadTalker = False #@param{type: "boolean"}
#@markdown #### Regional Prompter 설명 [영상 보기](https://youtu.be/wDk-BQnSBm0)
Regional_Prompter = False #@param{type: "boolean"}
Ultimate_SD_Upscale = False #@param{type: "boolean"}
Openpose_Editor = False #@param{type: "boolean"}
#@markdown #### ADetailer 설명 [영상 보기](https://youtu.be/axxmY8c3OGg)
ADetailer = False #@param{type: "boolean"}

#@markdown #### AnimateDiff 설명 [영상 보기1](https://youtu.be/BHMCSV6pRgM) [영상 보기2](https://youtu.be/8NXdx7SSuDc) [영상 보기3 (ADetailer포함)](https://youtu.be/axxmY8c3OGg)
AnimateDiff = False #@param{type: "boolean"}
DynamicThresholding = False #@param{type: "boolean"}
#@markdown #### DWPose는 최신 ControlNet에 이미 포함되어 있습니다. 확장 프로그램 설치가 필요없습니다. 위에 ControlNet을 선택하세요.
#@markdown #### DWPose 설명 [영상 보기](https://youtu.be/wVOTQWHiqpA)
DWPose = 'ControlNet에 이미 들어 있습니다' #@param ['ControlNet에 이미 들어 있습니다', 'ControlNet에 이미 들어 있다니까요!', '허허~ 거참 ControlNet을 선택하라구요~', '아~ 이사람 한 고집 하네~']
#@markdown #### Reactor 설명 [영상 보기](https://youtu.be/yfdbAqxyn14)
Reactor = False #@param{type: "boolean"}
#@markdown #### InpaintAnything 설명 [영상 보기](https://youtu.be/g8cTsK5cHWE)
InpaintAnything = False #@param{type: "boolean"}
#@markdown #### Replacer 설명 [영상 보기](https://youtu.be/C1I4i5pbidc)
Replacer = False #@param{type: "boolean"}

#@markdown ## 🦠🦠🦠🦠🦠🦠🦠🦠🦠🦠🦠🦠🦠🦠🦠🦠🦠🦠🦠🦠🦠🦠🦠🦠🦠
#@markdown ## 아래의 스크립트(Script)중에 사용할 모델을 선택하세요.
#@markdown ### 요청에 따라 모델을 추가하거나 변경할 수 있습니다.
#@markdown #### Loopback Wave 설명 [영상 보기](https://youtu.be/e8O0mUglIzk)
LoopbackWave = False #@param{type: "boolean"}

# #@markdown #### 3D Painting 설명 [영상 보기](https://youtu.be/6d0P07LZUH8)
# #@markdown ##### - 사용중 Depth map 생성에 상당한 시간이 걸립니다. (영상 참조)
# #@markdown ##### - 생성 과정은 아래의 output 창에서 볼수 있습니다.
# Depthmap_3D_Painting = True #@param{type: "boolean"}

# 로그 삭제
#Clear_Log = True #@param{type:'boolean'}

# #@markdown ## ➿➿➿➿➿➿➿➿➿➿➿➿➿➿➿➿➿➿➿➿➿➿➿➿➿
# #@markdown ## 위에 있는 Checkpoint 모델 외에 사용하고 싶은 모델의 다운로드 링크를 넣어주세요.
# #@markdown ### Hugging Face 와 civitAI만 가능합니다.
# #@markdown ### 각각의 링크 주소는 쉼표(,)로 구분해주세요.
# Model_URL = '' #@param {type: "string"}

# #@markdown ## ☎️☎️☎️☎️☎️☎️☎️☎️☎️☎️☎️☎️☎️☎️☎️☎️☎️☎️☎️☎️☎️☎️☎️☎️☎️
# #@markdown ## 위에 있는 확장 프로그램(Extensions) 외에 사용하고 싶은 모델의 URL을 넣어주세요.
# #@markdown ### GitHub만 가능합니다.
# #@markdown ### 각각의 URL 주소는 쉼표(,)로 구분해주세요.
# Extensions_URL = '' #@param {type: "string"}

# #@markdown ## 🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨🔨
# #@markdown ## 'COMMANDLINE_ARGS=' 에 추가할 값이 있다면 이곳에 넣어주세요.
# Extra_args = '' #@param {type: "string"}

if is_drive_used:
  from google.colab import drive
  drive.mount('/content/drive')
  output_path = '/content/drive/MyDrive/' + output_path
  root = '/content/'
  !mkdir -p {output_path}/outputs
  !mkdir -p {output_path}/checkpoints
  !mkdir -p {output_path}/ESRGAN
  !mkdir -p {output_path}/hypernetworks
else:
  # Don't connect to google drive
  output_path = '/content/' + output_path # Is it still needed?
  root = '/content/'

def print_message(msg):
  # print('📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍')
  # print('📍📍📍 ', msg, ' 📍📍📍')
  # print('📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍')
  pass

def clear():
    from IPython.display import clear_output
    return clear_output()

def downloadModel(url):
  if 'huggingface.co' in url:
    filename = url.split('/')[-1]
    filename = filename.removesuffix('?download=true')
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}  -o {filename}
  elif 'civitai.com' in url: # CivitAP는 더이상 사용 안합니다.
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}
  else:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}
    print_message('지원 하지 않는 URL이지만, 다운로드를 시도했습니다. 확인해 보시기 바랍니다.')

def checkpoint_exists(): ### See if a checkpoint model exists in goodle drive ###
  files = os.listdir(output_path + '/checkpoints/') # get all files
  for file in files: # check .ckpt or .safetensors extension
    if file.endswith('.ckpt') or file.endswith('.safetensors'):
      return True
  return False # no checkpoint file

def download_models():
  # Checkpoint 모델 다운로드
  models_count = 0
  %cd {root}/stable-diffusion-webui/models/Stable-diffusion
  print_message('사용할 모델을 다운받고 있습니다.')
  if RevAnimated:
    downloadModel('https://huggingface.co/fp16-guy/ReV_Animated_fp16_cleaned/resolve/main/revAnimated_v122_fp16.safetensors')
    models_count += 1
  if DreamShaper_8:
    downloadModel('https://huggingface.co/Lykon/DreamShaper/resolve/main/DreamShaper_8_pruned.safetensors')
    models_count += 1
  if DreamShaper_8_INPAINTING:
    downloadModel('https://huggingface.co/Lykon/DreamShaper/resolve/main/DreamShaper_8_INPAINTING.inpainting.safetensors')
    models_count += 1
  if PicX_real:
    downloadModel('https://huggingface.co/fp16-guy/PicX_real/resolve/main/picX_real.safetensors')
    models_count += 1
  if CyberRealistic_V41:
    downloadModel('https://huggingface.co/cyberdelia/CyberRealistic/resolve/main/CyberRealistic_V4.1_FP16.safetensors')
    models_count += 1
  if Realistic_Vision_V60:
    downloadModel('https://huggingface.co/SG161222/Realistic_Vision_V6.0_B1_noVAE/resolve/main/Realistic_Vision_V6.0_NV_B1_fp16.safetensors')
    models_count += 1
  if SDXL_10_Base_AND_Refiner:
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors')
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors')
    models_count += 1
  if Juggernaut_XL_v9_RunDiffusionPhoto_v2:
    downloadModel('https://huggingface.co/RunDiffusion/Juggernaut-XL-v9/resolve/main/Juggernaut-XL_v9_RunDiffusionPhoto_v2.safetensors')
    models_count += 1
  if Juggernaut_RunDiffusionPhoto2_Lightning_4Steps:
    downloadModel('https://huggingface.co/RunDiffusion/Juggernaut-XL-Lightning/resolve/main/Juggernaut_RunDiffusionPhoto2_Lightning_4Steps.safetensors')
    models_count += 1
  if Model_URL:
      for m in Model_URL.split(','):
        %cd {root}/stable-diffusion-webui/models/Stable-diffusion
        downloadModel(m)

  # 사용할 모델이 없으면, SD 1.5모델은 필수로 다운로드
  if models_count == 0 and not checkpoint_exists():
    downloadModel('https://huggingface.co/Lykon/DreamShaper/resolve/main/DreamShaper_8_pruned.safetensors')
    models_count += 1
  print_message(str(models_count) + '개의 Checkpoint 모델을 다운 받았습니다.')

  # LoRA 모델 다운로드
  %cd {root}/stable-diffusion-webui/models/Lora
  if DetailTweakerLoRA:
    downloadModel('https://huggingface.co/OedoSoldier/detail-tweaker-lora/resolve/main/add_detail.safetensors')
  if KoreanDollLikeness_V20:
    downloadModel('https://huggingface.co/Kanbara/doll-likeness-series/resolve/main/koreanDollLikeness_v20.safetensors')
  if LCM_LoRA_SD15:
    downloadModel('https://huggingface.co/latent-consistency/lcm-lora-sdv1-5/resolve/main/pytorch_lora_weights.safetensors')
  if Model_URL:
      for m in Model_URL.split(','):
        %cd {root}/stable-diffusion-webui/models/Lora
        downloadModel(m)

  # download VAEs - is it required???
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-ema-original/resolve/main/vae-ft-ema-560000-ema-pruned.ckpt -d {root}/stable-diffusion-webui/models/VAE -o vae-ft-ema-560000-ema-pruned.ckpt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d {root}/stable-diffusion-webui/models/VAE -o vae-ft-mse-840000-ema-pruned.ckpt

def install_extensions():
  # extensions directory
  %cd {root}/stable-diffusion-webui/extensions

  if ControlNet:
    !git clone https://github.com/Mikubill/sd-webui-controlnet.git
    %cd {root}/stable-diffusion-webui/extensions/sd-webui-controlnet/models
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime.pth')
    downloadModel('https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth')
    downloadModel('https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth')
    # brightness model
    downloadModel('https://huggingface.co/ioclab/ioc-controlnet/resolve/main/models/control_v1p_sd15_brightness.safetensors')
    # QR monster model
    downloadModel('https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/resolve/main/control_v1p_sd15_qrcode_monster.safetensors')
    downloadModel('https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/resolve/main/v2/control_v1p_sd15_qrcode_monster_v2.safetensors')
    downloadModel('https://huggingface.co/Nacholmo/controlnet-qr-pattern-sdxl/resolve/main/automatic1111/control_v01u_sdxl_qrpattern.safetensors')
    # SDXL model
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_canny_full.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_depth_full.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ip-adapter_xl.pth')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_blur.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_blur_anime.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_scribble_anime.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_recolor_256lora.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_sketch_256lora.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_diffusers_xl_lineart.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/thibaud_xl_openpose.safetensors')
    # insightface for IP Adapter (faceID)
    !pip install insightface
    %cd {root}/stable-diffusion-webui/extensions
  if Deforum:
    !git clone https://github.com/deforum-art/deforum-for-automatic1111-webui
    !apt-get install libvulkan1
  if SadTalker:
#    !git clone https://github.com/OpenTalker/SadTalker
#    !mkdir -p {root}/stable-diffusion-webui/models/SadTalker
#    %cd {root}/stable-diffusion-webui/models/SadTalker
#    url = 'https://huggingface.co/digiclau/files/resolve/main/extensions/SadTalker/sadtalker_checkpoints-20230517T015718Z.zip'
#    filename = "sadtalker_checkpoints.zip"
#    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url} -o {filename}
#    !unzip {filename}
#    !rm {filename}
#    # extra code to fix error
#    !mkdir /tmp
#    !mkdir /tmp/gradio
#    %cd {root}/stable-diffusion-webui/extensions
    !git clone https://github.com/OpenTalker/SadTalker
    !mkdir -p {root}/stable-diffusion-webui/models/SadTalker
    %cd {root}/stable-diffusion-webui/models/SadTalker
    url = 'https://huggingface.co/digiclau/files/resolve/main/extensions/SadTalker/sadtalker_checkpoints.zip'
    filename = "sadtalker_checkpoints.zip"
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url} -o {filename}
    !unzip {filename}
    !rm {filename}
#    # extra code to fix error
    !mkdir /tmp
    !mkdir /tmp/gradio
    %cd {root}/stable-diffusion-webui/extensions
  if Regional_Prompter:
    !git clone https://github.com/hako-mikan/sd-webui-regional-prompter
  if Ultimate_SD_Upscale:
    !git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111
  if Openpose_Editor:
    !git clone https://github.com/fkunn1326/openpose-editor
  if ADetailer:
    !git clone https://github.com/Bing-su/adetailer
  if AnimateDiff:
    !git clone https://github.com/continue-revolution/sd-webui-animatediff
    %cd {root}/stable-diffusion-webui/extensions/sd-webui-animatediff/model
    downloadModel('https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd14_v1.safetensors')
    downloadModel('https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd15_v1.safetensors')
    downloadModel('https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd15_v2.safetensors')
    downloadModel('https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd15_v3.safetensors')
    downloadModel('https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sdxl_hs.safetensors')
    downloadModel('https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sdxl_v10_beta.safetensors')

    downloadModel('https://huggingface.co/manshoety/AD_Stabilized_Motion/resolve/main/mm-Stabilized_high.pth')
    downloadModel('https://huggingface.co/manshoety/AD_Stabilized_Motion/resolve/main/mm-Stabilized_mid.pth')
    downloadModel('https://huggingface.co/manshoety/beta_testing_models/resolve/main/mm-p_0.5.pth')
    downloadModel('https://huggingface.co/manshoety/beta_testing_models/resolve/main/mm-p_0.75.pth')
    downloadModel('https://huggingface.co/CiaraRowles/TemporalDiff/resolve/main/temporaldiff-v1-animatediff.safetensors')

#    downloadModel('https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/lora/mm_sd15_v3_adapter.safetensors')
    %cd {root}/stable-diffusion-webui/extensions
  if DynamicThresholding:
    !git clone https://github.com/mcmonkeyprojects/sd-dynamic-thresholding
#  if DWPose:
#    !git clone https://github.com/IDEA-Research/DWPose
  if Reactor:
    !git clone https://github.com/Gourieff/sd-webui-reactor.git
#  if Depthmap_3D_Painting:
#    !git clone https://github.com/thygate/stable-diffusion-webui-depthmap-script
  if InpaintAnything:
    !git clone https://github.com/Uminosachi/sd-webui-inpaint-anything
  if Replacer:
    !git clone https://github.com/continue-revolution/sd-webui-segment-anything
    !git clone https://github.com/light-and-ray/sd-webui-replacer
    %cd {root}/stable-diffusion-webui/extensions/sd-webui-segment-anything/models/sam
    downloadModel('https://huggingface.co/lkeab/hq-sam/resolve/main/sam_hq_vit_h.pth')
    %cd {root}/stable-diffusion-webui/extensions

def install_scripts():
  if LoopbackWave:
    %cd {root}/stable-diffusion-webui/scripts
#    url = 'https://files.catbox.moe/0hx51x.py'
#    filename = 'loopback_wave.py'
#    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}  -o {filename}
    url = 'https://github.com/DigiClau/stablediffusion_webui/raw/main/scripts/loopback_wave.py'
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}

def updatePython():
  !python --version > /content/pyversion
  with open('/content/pyversion', 'r') as file:
      if '3.10' in file.read():
        print_message('이미 python 3.10이 설치되어 있습니다.')
        return

  #install python 3.10.6
  !apt-get update -y
  !apt-get install python3.10.6

  #change alternatives
  !rm /usr/local/bin/python
  !rm /usr/local/bin/pip
  !sudo apt-get install python3.10-distutils
  !sudo update-alternatives --install /usr/local/bin/python python /usr/bin/python3.10 2
  !wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py

def initSaveGoogleDriveModelOnly():
  # Use config file in google drive
  if not os.path.exists(output_path + '/ui-config.json'): #(only once at the first run)
    !wget https://raw.githubusercontent.com/DigiClau/stablediffusion_webui/main/ui-config.json -O {output_path + '/ui-config.json'}
  if not os.path.exists(output_path + '/config.json'): #(only once at the first run)
    !wget https://raw.githubusercontent.com/DigiClau/stablediffusion_webui/main/config.json -O {output_path + '/config.json'}
  if not os.path.exists(output_path + '/styles.csv'): #(only once at the first run)
    print("Create new styles.csv file.")
    !wget https://raw.githubusercontent.com/DigiClau/stablediffusion_webui/main/style.csv -O {output_path + '/styles.csv'}

  #!rm /content/stable-diffusion-webui/ui-config.json
  !ln -s {output_path + '/ui-config.json'} {root}/stable-diffusion-webui/
  !ln -s {output_path + '/config.json'} {root}/stable-diffusion-webui/
  !ln -s {output_path + '/styles.csv'} {root}/stable-diffusion-webui/
  !ln -s {output_path}/outputs # "%cd {root}/stable-diffusion-webui" command in the main

  # embeddings folder on Google Drive
  !mkdir -p {output_path}/embeddings
  !rm -rf embeddings # -rf r(recursive-delete subdir), f(force-no confirmation required)
  !ln -s {output_path}/embeddings

  # save parameter file in google drive
  if not os.path.exists(output_path + '/params.txt'): #(only once at the first run)
    !touch {output_path + '/params.txt'}
  !ln -s {output_path}/params.txt

  # link all checkpoints in the checkpoints folder
  %cd {root}/stable-diffusion-webui/models/Stable-diffusion
  checkpoints_in_google_drive = glob.glob(output_path + '/checkpoints/*')
  print_message('구글 드라이브에 들어 있는 Checkpoint 모델: %s'%checkpoints_in_google_drive)
  for f in checkpoints_in_google_drive:
    !ln -s {f}

  # link ESRGAN upscalers in the model folder
  !mkdir -p {root}/stable-diffusion-webui/models/ESRGAN
  %cd {root}/stable-diffusion-webui/models/ESRGAN
  upscalers_in_google_drive = glob.glob(output_path + '/ESRGAN/*')
  print_message('구글 드라이브에 들어 있는 Upscaler 모델: %s'%upscalers_in_google_drive)
  for f in upscalers_in_google_drive:
    !ln -s {f}

  # link RealESRGAN upscalers in the model folder
  !mkdir -p {root}/stable-diffusion-webui/models/RealESRGAN
  %cd {root}/stable-diffusion-webui/models
  !rm -rf RealESRGAN
  !ln -s {output_path}/RealESRGAN

  # link SwinIR upscalers in the model folder
  !mkdir -p {root}/stable-diffusion-webui/models/SwinIR
  %cd {root}/stable-diffusion-webui/models
  !rm -rf SwinIR
  !ln -s {output_path}/SwinIR

  # use lora model folder in google drive
  !mkdir -p {output_path}/Lora
  %cd {root}/stable-diffusion-webui/models
  !rm -rf Lora
  !ln -s {output_path}/Lora

# use LyCORIS model folder in google drive
  !mkdir -p {output_path}/Lora
  %cd {root}/stable-diffusion-webui/models
  !rm -rf LyCORIS
  !ln -s {output_path}/LyCORIS

  # use hypernetwork folder in google drive
  !mkdir -p {output_path}/hypernetworks
  %cd {root}/stable-diffusion-webui/models
  !rm -rf hypernetworks
  !ln -s {output_path}/hypernetworks

  # use VAE folder in google rive
  !mkdir -p {output_path}/VAE
  %cd {root}/stable-diffusion-webui/models
  !rm -rf VAE
  !ln -s {output_path}/VAE

  # use ControlNet folder in google rive
  !mkdir -p {output_path}/ControlNet
  %cd {root}/stable-diffusion-webui/models
  !rm -rf ControlNet
  !ln -s {output_path}/ControlNet

#def installExtensionsFromURL(urls):
#  %cd {root}/stable-diffusion-webui/extensions
#  for url in urls.split(','):
#    !git clone {url}

#def lowRamPatch():
#  !sed -i 's/dict()))$/dict())).cuda()/g'  {root}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py

updatePython()
!mkdir -p {root}
%cd {root}
# os.chdir(root)
!apt-get -y install -qq aria2
!pip install pyngrok
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui

# A1111 first launch
%cd {root}/stable-diffusion-webui
# ver='v1.6.0'
# !git checkout -f {ver}

if version == '1.7.0':
  !git checkout -f cf2772fab0af5573da775e7437e6acdca424f26e # 1.7.0
if version == '1.8.0':
  !git checkout -f bef51aed032c0aaa5cfd80445bc4cf0d85b408b5 # 1.8.0
!COMMANDLINE_ARGS="--exit"  python launch.py

if is_drive_used: # == GDriveSaveMode.ON:
  initSaveGoogleDriveModelOnly()

download_models()
install_extensions()
install_scripts()

# downgrade httpx to avoid TypeError: AsyncConnectionPool.__init__() got an unexpected keyword argument 'socket_options'
!pip install httpx==0.24.1

# clear()
# lowRamPatch()

%cd {root}/stable-diffusion-webui
#args = f'--gradio-img2img-tool color-sketch --enable-insecure-extension-access --gradio-queue --xformers'
args = f'--gradio-img2img-tool color-sketch --enable-insecure-extension-access --gradio-queue'

if ngrok:
  args += f' --ngrok {ngrok} '
else:
  args += ' --share '

### since 1.6.0, SDP attention instead of xformers
# extra_arguments = '--disable-model-loading-ram-optimization --opt-sdp-no-mem-attention'
extra_arguments = '--disable-model-loading-ram-optimization --opt-sdp-attention --medvram-sdxl'
#extra_arguments += ' --reinstall-torch' # this commandline flag to reinstall Torch 2.1.2 (by defult: 2.1.0)
args += ' ' + extra_arguments

#print(f'WEBUI ARGUMENTS: {args}')
!python {root}/stable-diffusion-webui/launch.py {args}